<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GOOGLEMAPS_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Import necessary libraries
import os
from google import genai
from google.genai import types

# --- 1. SETUP: Use Colab Secrets and AgentConfig (Remains unchanged) ---
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except ImportError:
    GOOGLE_API_KEY = os.environ.get('GEMINI_API_KEY')
    if not GOOGLE_API_KEY:
        raise ValueError("GEMINI_API_KEY not found. Please set the environment variable.")

os.environ['GEMINI_API_KEY'] = GOOGLE_API_KEY

class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-2.5-flash"

# --- 2. DEFINE QUERY AND CLIENT (CRITICAL CHANGE HERE) ---
client = genai.Client()
MODEL_NAME = AgentConfig.LLM_MODEL_NAME

# NEW QUERY: Forces the model to explicitly output a "REASONING PLAN" section
# before the final answer, simulating the agent's "Think & Plan" step.
GEOSPATIAL_QUERY_STRUCTURED = (
    "As a specialized Geospatial Reasoning Agent, first outline your steps under "
    "the heading '## REASONING PLAN' and then provide the final, grounded answer. "
    "Query: I need three highly-rated, affordable family restaurants near the Museum of Science in Boston. "
    "For each, provide the name, address, and a brief summary of why the Maps tool recommends it."
)


def run_geospatial_query_structured(client: genai.Client, prompt: str):
    """
    Runs the prompt using a structured format to force explicit, step-by-step reasoning.
    """
    print(f"\n🤖 Calling {MODEL_NAME} with Maps Grounding and Structured Prompt...")
    print("Expected Output: Explicit REASONING PLAN followed by the FINAL ANSWER.")
    print("-" * 70)

    config = types.GenerateContentConfig(
        tools=[
            types.Tool(google_maps=types.GoogleMaps()) # Enables Maps Grounding
        ]
    )

    # Call the model
    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=prompt,
        config=config
    )

    # --- 3. DISPLAY FULL, STRUCTURED RESPONSE ---
    print("✅ Full Structured Response (includes forced reasoning plan):")
    print(response.text)

    # --- 4. CITATION VERIFICATION (The original check remains reliable for most errors) ---
    maps_citations = []
    grounding_metadata = getattr(getattr(response.candidates[0], 'grounding_metadata', None), 'grounding_chunks', [])

    if grounding_metadata:
        for chunk in grounding_metadata:
            uri = getattr(getattr(chunk, 'web', None), 'uri', '')
            # A rough check for the content URI pattern (often uses 'googleusercontent' or 'maps.google.com')
            if 'maps.google.com' in uri or 'googleusercontent.com/maps.google.com/4' in uri:
                maps_citations.append(getattr(chunk.web, 'title', 'Maps Source'))

    if maps_citations:
        print("\n\n📍 Response Source Evidence (Successful Grounding):")
        print("The response successfully utilized the Maps tool to retrieve data for the analysis.")
    else:
        print("\n\n⚠️ Source Evidence: The model performed the task but explicit Maps grounding metadata was not captured for citation.")


# --- 5. EXECUTION ---
try:
    # Use the structured query function
    run_geospatial_query_structured(client, GEOSPATIAL_QUERY_STRUCTURED)
except Exception as e:
    print(f"\nAn unhandled error occurred: {e}")
    print("Please check connectivity, API key validity, and library installation.")

Google Generative AI configured successfully using Colab Secrets.

🤖 Calling gemini-2.5-flash with Maps Grounding and Structured Prompt...
Expected Output: Explicit REASONING PLAN followed by the FINAL ANSWER.
----------------------------------------------------------------------
✅ Full Structured Response (includes forced reasoning plan):
## REASONING PLAN

1. **Search for restaurants:** Use Google Maps to search for "highly-rated affordable family restaurants near Museum of Science in Boston".
2. **Filter and select:** From the search results, identify three suitable restaurants that are highly-rated, affordable, and family-friendly.
3. **Extract information:** For each selected restaurant, extract its name, address, and a brief summary of why it's recommended based on the search result snippets.
4. **Format the answer:** Present the information clearly, with citations for each restaurant.

Here are three highly-rated, affordable family restaurants near the Museum of Science in Bosto

In [14]:
!pip install folium geopy -q

In [15]:
# Import necessary libraries
import os
import urllib.parse
from google import genai
from google.genai import types

# New imports for map visualization
import folium
# Need to import 'display' to show the map in Colab/Jupyter
from IPython.display import display
from geopy.geocoders import Nominatim

# --- 1. SETUP: Use Colab Secrets and AgentConfig ---
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except ImportError:
    GOOGLE_API_KEY = os.environ.get('GEMINI_API_KEY')
    if not GOOGLE_API_KEY:
        raise ValueError("GEMINI_API_KEY not found. Please set the environment variable.")

os.environ['GEMINI_API_KEY'] = GOOGLE_API_KEY

class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-2.5-flash"

# --- 2. DEFINE QUERY AND CLIENT ---
client = genai.Client()
MODEL_NAME = AgentConfig.LLM_MODEL_NAME

# Query: Forces the model to explicitly output a "REASONING PLAN" section
GEOSPATIAL_QUERY_STRUCTURED = (
    "As a specialized Geospatial Reasoning Agent, first outline your steps under "
    "the heading '## REASONING PLAN' and then provide the final, grounded answer. "
    "Query: I need three highly-rated, affordable family restaurants near the Museum of Science in Boston. "
    "For each, provide the name, address, and a brief summary of why the Maps tool recommends it. "
    "Ensure the final answer clearly lists the name and exact street address for easy plotting."
)


def run_geospatial_query_structured(client: genai.Client, prompt: str):
    """
    Runs the prompt, then plots the results on a Folium map.
    """
    print(f"\n🤖 Calling {MODEL_NAME} with Maps Grounding and Structured Prompt...")
    print("Expected Output: Explicit REASONING PLAN followed by the FINAL ANSWER.")
    print("-" * 70)

    config = types.GenerateContentConfig(
        tools=[
            types.Tool(google_maps=types.GoogleMaps()) # Enables Maps Grounding
        ]
    )

    # Call the model
    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=prompt,
        config=config
    )

    # --- 3. DISPLAY FULL, STRUCTURED RESPONSE ---
    print("✅ Full Structured Response (includes forced reasoning plan):")
    print(response.text)

    # --- 4. MAP GENERATION LOGIC (Corrected for Geocoding) ---
    print("\n" * 2)
    print("🗺️ GENERATING INTERACTIVE MAP...")

    # *** CORRECTED LOCATIONS LIST ***
    # We use simple street addresses for better geocoding success with Nominatim.
    locations_to_plot = [
        ("Museum of Science, Boston, MA", "Museum of Science (Start)", "blue"),
        # Simplified addresses below to avoid geocoding errors like "Could not find coordinates"
        ("162 Salem St, Boston, MA", "Theo's Cozy Corner Restaurant (4.7 ⭐)", "green"),
        ("141 Salem St, Boston, MA", "Dino's Cafe (4.7 ⭐)", "green"),
        ("80 Causeway St, Boston, MA", "Hub Hall (4.2 ⭐)", "green")
    ]

    # Initialize geolocator (using a unique user_agent is required by Nominatim)
    geolocator = Nominatim(user_agent="geospatial_agent_map_plotter_v2")

    # Find the Museum's coordinates to center the map
    museum_location = geolocator.geocode("Museum of Science, Boston, MA", timeout=10)
    if not museum_location:
        print("⚠️ Could not locate Museum of Science to center map. Map generation aborted.")
        return

    # Create the base map centered on the museum
    m = folium.Map(location=[museum_location.latitude, museum_location.longitude], zoom_start=14)

    # Add markers for all locations
    for full_address, name, color in locations_to_plot:
        try:
            # Use a timeout to prevent hanging on a failed lookup
            location = geolocator.geocode(full_address, timeout=10)
            if location:
                folium.Marker(
                    location=[location.latitude, location.longitude],
                    popup=name,
                    tooltip=name, # Add a tooltip for hovering
                    icon=folium.Icon(color=color, icon='cutlery' if color == 'green' else 'star')
                ).add_to(m)
                print(f"✅ Plotted: {name}")
            else:
                print(f"⚠️ Could not find coordinates for: {name}")
        except Exception as e:
             # Catch specific connection/timeout errors
             print(f"❌ Geocoding error for {name}: {e}")

    # Display the map in the notebook
    print("\nDisplaying Map:")
    display(m)

    # --- 5. CITATION VERIFICATION (Original Check) ---
    maps_citations = []
    grounding_metadata = getattr(getattr(response.candidates[0], 'grounding_metadata', None), 'grounding_chunks', [])

    if grounding_metadata:
        for chunk in grounding_metadata:
            uri = getattr(getattr(chunk, 'web', None), 'uri', '')
            if 'http://googleusercontent.com/maps.google.com' in uri:
                maps_citations.append(getattr(chunk.web, 'title', 'Maps Source'))

    if maps_citations:
        print("\n\n📍 Response Source Evidence (Successful Grounding):")
        print("The response successfully utilized the Maps tool to retrieve data for the analysis.")
    else:
        print("\n\n⚠️ Source Evidence: The model performed the task but explicit Maps grounding metadata was not captured for citation.")


# --- 6. EXECUTION ---
try:
    # Use the structured query function
    run_geospatial_query_structured(client, GEOSPATIAL_QUERY_STRUCTURED)
except Exception as e:
    print(f"\nAn unhandled error occurred: {e}")
    print("Please check connectivity, API key validity, and library installation.")

Google Generative AI configured successfully using Colab Secrets.

🤖 Calling gemini-2.5-flash with Maps Grounding and Structured Prompt...
Expected Output: Explicit REASONING PLAN followed by the FINAL ANSWER.
----------------------------------------------------------------------
✅ Full Structured Response (includes forced reasoning plan):
## REASONING PLAN

1. **Search:** Use the `google_maps` tool to search for "highly-rated affordable family restaurants near Museum of Science in Boston".
2. **Filter and Select:** From the search results, identify the top three highly-rated and affordable family restaurants. I will prioritize results that explicitly mention family-friendliness, good reviews (high star rating, many reviews), and a reasonable price level.
3. **Extract Information:** For each of the selected restaurants, extract the following details: name, full address, and a brief summary of why it's recommended, based on the provided snippets (e.g., specific features, popular dishes,



⚠️ Source Evidence: The model performed the task but explicit Maps grounding metadata was not captured for citation.
